# 🔥🔥 Evaluate DGCNN Model Weights & Biases 🪄🐝

<!--- @wandbcode{pyg-dgcnn-eval} -->

This notebook demonstrates the evaluation of [Dynamic Graph CNN](https://arxiv.org/pdf/1801.07829.pdf) for point cloud segmnetation. You can check the following notebook for referring to the training code:

[![](https://colab.research.google.com/assets/colab-badge.svg)]()

# Install Required Packages

In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

In [ ]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install -q wandb

## Import Libraries

In [1]:
import os
import wandb
import random
import numpy as np
from tqdm.auto import tqdm

import torch
import torch.nn.functional as F

from torch_scatter import scatter
from torchmetrics.functional import jaccard_index

import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MLP, DynamicEdgeConv

# Initialize Weights & Biases

We need to call [`wandb.init()`](https://docs.wandb.ai/ref/python/init) once at the beginning of our program to initialize a new job. This creates a new run in W&B and launches a background process to sync data.

In [2]:
wandb_project = "pyg-point-cloud" #@param {"type": "string"}
wandb_run_name = "evaluate-dgcnn" #@param {"type": "string"}

wandb.init(project=wandb_project, name=wandb_run_name, job_type="evaluate")

config = wandb.config

config.seed = 42
config.device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(config.seed)
torch.manual_seed(config.seed)
device = torch.device(config.device)

config.category = 'Airplane' #@param ["Bag", "Cap", "Car", "Chair", "Earphone", "Guitar", "Knife", "Lamp", "Laptop", "Motorbike", "Mug", "Pistol", "Rocket", "Skateboard", "Table"] {type:"raw"}
config.random_jitter_translation = 1e-2
config.random_rotation_interval_x = 15
config.random_rotation_interval_y = 15
config.random_rotation_interval_z = 15
config.batch_size = 1
config.num_workers = 6

config.num_nearest_neighbours = 30
config.aggregation_operator = "max"
config.dropout = 0.5
config.initial_lr = 1e-3
config.lr_scheduler_step_size = 20
config.gamma = 0.8

config.artifact_address = 'wandb/point-cloud-segmentation/dgcnn-3n97rfrv-checkpoint:v29'
config.epochs = 30

wandb: Currently logged in as: geekyrakshit (wandb). Use `wandb login --relogin` to force relogin


# Load ShapeNet Dataset using PyTorch Geometric

We now load, preprocess and batch the ModelNet dataset for training, validation/testing and visualization.

In [3]:
transform = T.Compose([
    T.RandomJitter(config.random_jitter_translation),
    T.RandomRotate(config.random_rotation_interval_x, axis=0),
    T.RandomRotate(config.random_rotation_interval_y, axis=1),
    T.RandomRotate(config.random_rotation_interval_z, axis=2)
])
pre_transform = T.NormalizeScale()

In [4]:
dataset_path = os.path.join('ShapeNet', config.category)

train_dataset = ShapeNet(
    dataset_path, config.category, split='trainval',
    transform=transform, pre_transform=pre_transform
)
test_dataset = ShapeNet(
    dataset_path, config.category, split='test',
    pre_transform=pre_transform
)

In [5]:
segmentation_class_frequency = {}
for idx in tqdm(range(len(train_dataset))):
    pc_viz = train_dataset[idx].pos.numpy().tolist()
    segmentation_label = train_dataset[idx].y.numpy().tolist()
    for label in set(segmentation_label):
        segmentation_class_frequency[label] = segmentation_label.count(label)
for idx in tqdm(range(len(test_dataset))):
    pc_viz = train_dataset[idx].pos.numpy().tolist()
    segmentation_label = train_dataset[idx].y.numpy().tolist()
    for label in set(segmentation_label):
        segmentation_class_frequency[label] = segmentation_label.count(label)
class_offset = min(list(segmentation_class_frequency.keys()))
class_offset

  0%|          | 0/2349 [00:00<?, ?it/s]

  0%|          | 0/341 [00:00<?, ?it/s]

0

In [6]:
for idx in tqdm(range(len(train_dataset))):
    train_dataset[idx].y -= class_offset
for idx in tqdm(range(len(test_dataset))):
    test_dataset[idx].y -= class_offset

  0%|          | 0/2349 [00:00<?, ?it/s]

  0%|          | 0/341 [00:00<?, ?it/s]

In [7]:
train_loader = DataLoader(
    train_dataset, batch_size=config.batch_size,
    shuffle=True, num_workers=config.num_workers
)
test_loader = DataLoader(
    test_dataset, batch_size=config.batch_size,
    shuffle=False, num_workers=config.num_workers
)

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# Load Checkpoint

In [8]:
class DGCNN(torch.nn.Module):
    def __init__(self, out_channels, k=30, aggr='max'):
        super().__init__()

        self.conv1 = DynamicEdgeConv(MLP([2 * 6, 64, 64]), k, aggr)
        self.conv2 = DynamicEdgeConv(MLP([2 * 64, 64, 64]), k, aggr)
        self.conv3 = DynamicEdgeConv(MLP([2 * 64, 64, 64]), k, aggr)

        self.mlp = MLP(
            [3 * 64, 1024, 256, 128, out_channels],
            dropout=0.5, norm=None
        )

    def forward(self, data):
        x, pos, batch = data.x, data.pos, data.batch
        x0 = torch.cat([x, pos], dim=-1)
        
        x1 = self.conv1(x0, batch)
        x2 = self.conv2(x1, batch)
        x3 = self.conv3(x2, batch)
        
        out = self.mlp(torch.cat([x1, x2, x3], dim=1))
        return F.log_softmax(out, dim=1)

Since we saved the checkpoints as artifacts on our Weights & Biases workspace, we can now fetch and load them.

In [9]:
config.num_classes = train_dataset.num_classes

model = DGCNN(
    out_channels=train_dataset.num_classes,
    k=config.num_nearest_neighbours,
    aggr=config.aggregation_operator
).to(device)

model_artifact = wandb.use_artifact(config.artifact_address, type='checkpoint')
artifact_dir = model_artifact.download()
model_checkpoint_path = os.path.join(artifact_dir, "checkpoint.pt")

model.load_state_dict(torch.load(model_checkpoint_path)["model_state_dict"])

wandb:   1 of 1 files downloaded.  


<All keys matched successfully>

# Evaluation

In [10]:
def evaluate(loader, split, table):
    total_accuracy, total_iou = 0, 0
    for data in tqdm(loader):
        data = data.to(device)
        with torch.no_grad():
            model.eval()
            outs = model(data)

            predicted_labels = outs.argmax(dim=1)
            accuracy = predicted_labels.eq(data.y).sum().item() / data.num_nodes

            sizes = (data.ptr[1:] - data.ptr[:-1]).tolist()
            ious, categories = [], []
            y_map = torch.empty(
                loader.dataset.num_classes, device=device
            ).long()
            for out, y, category in zip(
                outs.split(sizes), data.y.split(sizes), data.category.tolist()
            ):
                category = list(ShapeNet.seg_classes.keys())[category]
                part = ShapeNet.seg_classes[category]
                part = torch.tensor(part, device=device)
                y_map[part] = torch.arange(part.size(0), device=device)
                iou = jaccard_index(
                    out[:, part].argmax(dim=-1), y_map[y],
                    task="multiclass", num_classes=part.size(0)
                )
                ious.append(iou)
            categories.append(data.category)
            iou = torch.tensor(ious, device=device)
            category = torch.cat(categories, dim=0)
            mean_iou = float(scatter(iou, category, reduce='mean').mean())

            gt_pc_viz = data.pos.cpu().numpy().tolist()
            segmentation_label = data.y.cpu().numpy().tolist()
            frequency_dict = {key: 0 for key in segmentation_class_frequency.keys()}
            for label in set(segmentation_label):
                frequency_dict[label] = segmentation_label.count(label)
            for j in range(len(gt_pc_viz)):
                gt_pc_viz[j] += [segmentation_label[j] + 1 - class_offset]

            predicted_pc_viz = data.pos.cpu().numpy().tolist()
            segmentation_label = data.y.cpu().numpy().tolist()
            frequency_dict = {key: 0 for key in segmentation_class_frequency.keys()}
            for label in set(segmentation_label):
                frequency_dict[label] = segmentation_label.count(label)
            for j in range(len(predicted_pc_viz)):
                predicted_pc_viz[j] += [segmentation_label[j] + 1 - class_offset]

            table.add_data(
                wandb.Object3D(np.array(gt_pc_viz)),
                wandb.Object3D(np.array(predicted_pc_viz)),
                accuracy, mean_iou, split, "DGCNN"
            )
            total_accuracy += accuracy
            total_iou += mean_iou
    
    wandb.log({
        f"{split}/Accuracy": total_accuracy / len(loader),
        f"{split}/IoU": total_iou / len(loader),
    })
    
    return table

We evaluate the results and store them in a Weights & Biases Table.

In [11]:
table = wandb.Table(columns=["Ground-Truth", "Prediction", "Accuracy", "IoU", "Split", "Model-Name"])
evaluate(train_loader, "Train-Val", table)
evaluate(test_loader, "Test", table)

  0%|          | 0/2349 [00:00<?, ?it/s]

  0%|          | 0/341 [00:00<?, ?it/s]

In [12]:
wandb.log({"Evaluation-Results": table})

In [13]:
wandb.finish()

wandb: Network error (ConnectTimeout), entering retry loop.
wandb: Network error (ConnectTimeout), entering retry loop.


Test/Accuracy,▁
Test/IoU,▁
Train-Val/Accuracy,▁
Train-Val/IoU,▁
Test/Accuracy,0.90377
Test/IoU,0.73049
Train-Val/Accuracy,0.92111
Train-Val/IoU,0.80482
